#**CUBERTA, A NEURAL NETWORK FOR US STOCK ANALYSIS**  
---  
  
## A brief background:
As a young computer science student and a [value investor][1] in the stock market, I always wondered if at some point I could do some research to fuse those different worlds in a fresh new project. Thats when I bumped into the world of machine learning, then deep learning, and now Neural Networks. Let me introduce you to the new predicting model: *Cuberta*. 

## Project Goals:
*Cuberta* consist in a multi-layered Neural Network that looks forward predicting the average stock price growth and the [dividend yield][2] of a company the next fiscal year, based on a series of fundamental stock analysis ratios.
In order to keep everyone in track, lets leave an example:  

In 2019, IBM had an average price of 126\\$ per stock.  
In 2020, IBM had an average price of 114\\$ per stock, with a dividend yield of 0.052.
The aim of *Cuberta* is to predict (the Y, or Labels) the average stock price growth from 2019 to 2020, 114/126=0.9047 (which is actually degrowth) and a diviend yield of 0.052 per stock.

As many readers will know, using $X \in[-1, 1]$ rather than $X\in{\Bbb R}$ is, generally speaking, a good practice when working with this kind of algorithms, and since our  that's what we will aim for.

##Tools:
The libraries we will be using this project are:

1.   **TensorFlow**: a general purpose Machine Learning tool
2.   Keras: an open source, deep learning library written in Python.
3.   **NumPy**: a useful python library when working with more complex data/structure operations
4.   **MatPlotLib**: to give some visuals to the project
5.   **Pandas**: essential library when working with huge dataframes.

##Project structure:
The project will be split in 4 main parts:

1.   **Data extraction**: Build our dataframe using a couple of tools from the [AlphaVantage API][3]
2.   **Data Cleaning and Refactoring**: Refactor and convert the raw data from the API to the actual train and test DataFrames *Cuberta* will be given.
3.  **Fitting and evaluating the model**: Given the train and test DataFrames, training and testing the model to check it's accuracy.
4.  **Conclusions**: After *Cuberta* is built and ready,  drawing conclusions from the actual results


[1]: https://en.wikipedia.org/wiki/Value_investing
[2]: https://en.wikipedia.org/wiki/Dividend_yield
[3]: https://www.alphavantage.co/


In [4]:
# %tensorflow_version 2.x  # Execute only if you are on Colab


SyntaxError: invalid syntax (<ipython-input-4-29c5acef9024>, line 2)

In [1]:
# Some useful libs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests, json, csv
from io import StringIO
from IPython.display import clear_output, display
import shutil
import time
from datetime import datetime
import dateutil
import keras

NotFoundError: /casita/anaconda/anaconda3/lib/python3.8/site-packages/tensorflow/core/kernels/libtfkernel_sobol_op.so: undefined symbol: _ZN10tensorflow8OpKernel11TraceStringB5cxx11EPNS_15OpKernelContextEb

## 1. DATA EXTRACTION
we will get all the raw data from the alphavantage API

First let's create all the directories we will using:
Inside our ./companies directory, we will have 1 directory per symbol  

We will also need a metadata.json file to keep track of some inside info

Finally, set a delay for the API since its restricted to ~4 calls/min and 500 calls/day

In [22]:
try:
        os.mkdir("./companies/")
except:
        print("File already exists")

if not(os.path.exists("metadata.json")):
        metadata = {
                "last_stock_index": "0"
        }
        with open('metadata.json', 'w') as outfile:
                json.dump(metadata, outfile)

# Delay for each API call
ALPHA_VANTAGE_DELAY = 14

# Minimun years that a stock needs to have:
MINIMUN_DATA_YEARS = 3

# List of ratios we will be using
#TODO: añadir acciones_internas/acciones_totales
FINANCIAL_RATIOS_NAMES = ["Contribution_To_Debt", "ROE", "ROA", "Sales_Margin",         "ROIC", "ROCE", "EV/EBIT", "PBV", "PER", "Dividend_Yield","Treasury_Ratio", "Acid_Test", "RSCD"]

# The different API functions
API_FUNCTIONS = ["TIME_SERIES_MONTHLY_ADJUSTED", "INCOME_STATEMENT",
        "BALANCE_SHEET", "CASH_FLOW"]

File already exists


Define our API_KEY and the API main url  
Define functions to get all data from the API:

In [5]:
API_KEY = "428NS49PHMWU9PRB"
API_URL = "https://www.alphavantage.co/query?"

def download_symbol_raw(symbol, verbose = True):
    try:
        os.mkdir("./companies/" + symbol)
    except:
        if(verbose):
            print("File already exists")

    # Download symbol data:
    data = { "function": "",
        "symbol": symbol,
        "apikey": API_KEY
    }
    # Download each required csv from api:
    for func in API_FUNCTIONS:
        if(verbose):
            print('Downloading ' + func + '...')

        data["function"] = func
        request_content = requests.get(API_URL, data).json()
       
        # If we dont get a response for any function, we will skip this entire stock
        if (not('Monthly Time Series' in request_content) and 
            not('quarterlyReports' in request_content)):
            if(verbose):
                print("Error getting stock data")
            
            # Delete stock subfolder
            shutil.rmtree('./companies/' + symbol + '/', ignore_errors=True)
            return False

        # convert to a pd dataframe:
        if(func == "TIME_SERIES_MONTHLY"):
            key_word = "Monthly Time Series"
        else:
            key_word = "quarterlyReports"
        
        df = pd.DataFrame.from_dict(request_content[key_word])

        if(func == "TIME_SERIES_MONTHLY"):
            df = df.transpose()

        df.to_csv('./companies/' + symbol+ '/' + symbol + '_' + func + '.csv')

    print("Finished!")
    return True

def download_symbols_name_list(verbose = True):
    if(verbose):
        print("Downloading listing stocks on the USA market...")
    data = { "function": "LISTING_STATUS",
        "apikey": API_KEY 
    }
    response = requests.get(API_URL, data)
    data = StringIO(str(response.content).replace("\\r\\n", "_"))

    df = pd.read_csv(data, sep=",", lineterminator="_").set_index("b'symbol")
    # Match only stocks of assetType stock(We don't want to work with CFDs)
    df = df[df['assetType'] == "Stock"]

    df.to_csv('symbols_name_list.csv')

    print("Finished!")
    

    
def full_raw_download(verbose=True):
    # Check if file exists to save api call
    if not(os.path.exists('./symbols_name_list.csv')):
        download_symbols_name_list('symbols_name_list.csv')

    df = pd.read_csv('symbols_name_list.csv')["b'symbol"]


    metadata_file = open('metadata.json')
    last_index = int(json.load(metadata_file)['last_stock_index'])
    metadata_file.close()
    for i in range(last_index, len(df)):
        time.sleep(ALPHA_VANTAGE_DELAY*4) # Get ~1 ticket per minute
        # if download_symbol_raw() returns False, probably we cant do more api               calls for today. Save record of last stock we downloaded in metadata
        flag = download_symbol_raw(symbol=df[i], verbose=False)
        if (flag == False):
            print("Enough API calls. Stopping...")
            metadata = {
                "last_stock_index": str(i)
            }
            with open('metadata.json', 'w') as outfile:
                json.dump(metadata, outfile)

            return False

        else:
            print("Last symbol downloaded: " + df[i])

    return True
        


We now call the full_raw_download() function to start the extraction

In [6]:
#full_raw_download()

After finishing downloading the full raw data, we need to convert it to the ratios we will use to fit our model. For this purpose we will define some functions:

In [7]:
#TODO: si el patrimonio(commonStockTotalEquity) es negativo BORRAR LA ACCION PORQUE ES BASURA(NO ENTRENAR) Y Decirle a l programa q cuando reciba un dato asi directamente diga que es mala inversion
def get_Contributions_To_Debt(raw_dfs, i):
    interestExpense = raw_dfs["INCOME_STATEMENT"]["interestExpense"][i]
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    commonStockTotalEquity = raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"][i]
    longTermDebt = raw_dfs["BALANCE_SHEET"]["longTermDebt"][i]
    totalShareholderEquity = raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"][i]

    return (((interestExpense + netIncome)/(commonStockTotalEquity + longTermDebt)) 
        - ((netIncome)/(totalShareholderEquity)))

# can be any vale. if netIncome and totalShareholderEquity <0, this company is broke
def get_ROE(raw_dfs, i):
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    totalShareholderEquity = raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"][i]

    return ((netIncome)/(totalShareholderEquity - netIncome))

def get_ROA(raw_dfs, i):
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    totalAssets = raw_dfs["BALANCE_SHEET"]["totalAssets"][i]

    return((netIncome)/(totalAssets))

def get_Sales_Margin(raw_dfs, i):
    operatingIncome = raw_dfs["INCOME_STATEMENT"]["operatingIncome"][i]
    totalRevenue = raw_dfs["INCOME_STATEMENT"]["totalRevenue"][i]
    if (operatingIncome == 0 or totalRevenue == 0):
        return 0
    else:
        return (operatingIncome)/(totalRevenue)

def get_ROIC(raw_dfs, i):
    netIncomeApplicableToCommonShares = raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"][i]
    totalRevenue = raw_dfs["INCOME_STATEMENT"]["totalRevenue"][i]
    commonStockTotalEquity = raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"][i]
    longTermDebt = raw_dfs["BALANCE_SHEET"]["longTermDebt"][i]
    dividendPayout = raw_dfs["CASH_FLOW"]["dividendPayout"][i]

    return ((netIncomeApplicableToCommonShares + dividendPayout) /
        (commonStockTotalEquity + longTermDebt))

def get_ROCE(raw_dfs, i):
    commonStockTotalEquity = raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"][i]
    longTermDebt = raw_dfs["BALANCE_SHEET"]["longTermDebt"][i]
    ebit = raw_dfs["INCOME_STATEMENT"]["ebit"][i]

    return ( ebit / (longTermDebt + commonStockTotalEquity))

def get_EVpEBIT(raw_dfs, i):
    ebit = raw_dfs["INCOME_STATEMENT"]["ebit"][i]
    longTermDebt = raw_dfs["BALANCE_SHEET"]["longTermDebt"][i]   
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    average_price = raw_dfs["PRICES"]["averagePrice"][i]
    commonStockSharesOutstanding = raw_dfs["BALANCE_SHEET"]["commonStockSharesOutstanding"][i]
    cashAndShortTermInvestments = raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"][i]

    return((commonStockSharesOutstanding * average_price + longTermDebt - cashAndShortTermInvestments) / (ebit))
    
def get_PBV(raw_dfs, i):
    commonStockTotalEquity = raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"][i]
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    commonStockSharesOutstanding = raw_dfs["BALANCE_SHEET"]["commonStockSharesOutstanding"][i]
    average_price = raw_dfs["PRICES"]["averagePrice"][i]

    return((commonStockSharesOutstanding * average_price) / (commonStockTotalEquity))   

def get_PER(raw_dfs, i):
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    commonStockSharesOutstanding = raw_dfs["BALANCE_SHEET"]["commonStockSharesOutstanding"][i]
    average_price = raw_dfs["PRICES"]["averagePrice"][i]
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]

    return(commonStockSharesOutstanding* average_price)/(netIncome)

def get_Dividend_Yield(raw_dfs, i):
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    commonStockSharesOutstanding = raw_dfs["BALANCE_SHEET"]["commonStockSharesOutstanding"][i]
    average_price = raw_dfs["PRICES"]["averagePrice"][i]
    dividendPayout = raw_dfs["CASH_FLOW"]["dividendPayout"][i]

    if(dividendPayout == 0):
        return 0
    else:
        return((dividendPayout) / (commonStockSharesOutstanding* average_price))

def get_Treasury_Ratio(raw_dfs, i):
    cashAndShortTermInvestments = raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"][i]
    totalCurrentLiabilities = raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"][i]

    return((cashAndShortTermInvestments)/(totalCurrentLiabilities))

def get_Acid_Test(raw_dfs, i):
    totalCurrentAssets = raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"][i]
    inventory = raw_dfs["BALANCE_SHEET"]["inventory"][i]
    totalCurrentLiabilities = raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"][i]

    return((totalCurrentAssets - inventory) / totalCurrentLiabilities)

def get_RSCD(raw_dfs, i):
    ebit = raw_dfs["INCOME_STATEMENT"]["ebit"][i]
    interestExpense = raw_dfs["INCOME_STATEMENT"]["interestExpense"][i]
    shortTermDebt = raw_dfs["BALANCE_SHEET"]["shortTermDebt"][i]

    return((ebit) / (interestExpense + shortTermDebt))

# This is not directly used in the NN, but still useful data to have on the frame
def get_Enterprise_Value(raw_dfs, i):
    average_price = raw_dfs["PRICES"]["averagePrice"][i]
    longTermDebt = raw_dfs["BALANCE_SHEET"]["longTermDebt"][i]   
    netIncome = raw_dfs["INCOME_STATEMENT"]["netIncome"][i]
    average_price = raw_dfs["PRICES"]["averagePrice"][i]
    commonStockSharesOutstanding = raw_dfs["BALANCE_SHEET"]["commonStockSharesOutstanding"][i]
    cashAndShortTermInvestments = raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"][i]

    return (commonStockSharesOutstanding * average_price + longTermDebt - cashAndShortTermInvestments)

# used to calculate all ratios at once
RATIOS_FUNCTIONS = {
    "Contributions_To_Debt": get_Contributions_To_Debt,
    "ROE": get_ROE,
    "ROA": get_ROA,
    "Sales_Margin": get_Sales_Margin,         
    "ROIC": get_ROIC, 
    "ROCE": get_ROCE, 
    "EV/EBIT": get_EVpEBIT, 
    "PBV": get_PBV, 
    "PER": get_PER, 
    "Dividend_Yield": get_Dividend_Yield,
    "Treasury_Ratio": get_Treasury_Ratio, 
    "Acid_Test": get_Acid_Test, 
    "RSCD": get_RSCD
}

We define a function to refactor our annual data to a fitable dataset for the model

In [8]:
# TODO: terminar de hacer fill a los valores importantes, y checkear que hay algunos valores. Ej si falta el commonStockSharesOutstanding para calcular el EV, hay que tirar esa columna entera

# Returns the new raw_dfs with filled  and clean new values. Returns empty dataframe if data is too bad to be used
def fill_raw_data(raw_dfs) -> {}:
    empty = {}
    # interestExpense:
    if (raw_dfs["INCOME_STATEMENT"]["interestExpense"].isna().sum() > 1):
        return empty
    elif(raw_dfs["INCOME_STATEMENT"]["interestExpense"].isna().sum() == 1):
        raw_dfs["INCOME_STATEMENT"]["interestExpense"] = pd.concat([raw_dfs["INCOME_STATEMENT"]["interestExpense"].ffill(), raw_dfs["INCOME_STATEMENT"]["interestExpense"].bfill()]).groupby(level=0).mean()
    
    raw_dfs["INCOME_STATEMENT"]["interestExpense"] = raw_dfs["INCOME_STATEMENT"]["interestExpense"].abs()

    # netIncome:
    if (raw_dfs["INCOME_STATEMENT"]["netIncome"].isna().sum() > 1):
        return empty
    elif(raw_dfs["INCOME_STATEMENT"]["netIncome"].isna().sum() == 1):
        raw_dfs["INCOME_STATEMENT"]["netIncome"] = pd.concat([raw_dfs["INCOME_STATEMENT"]["netIncome"].ffill(), raw_dfs["INCOME_STATEMENT"]["netIncome"].bfill()]).groupby(level=0).mean()

    # commonStockTotalEquity:
    if (raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"] = pd.concat([raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"].ffill(), raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"].bfill()]).groupby(level=0).mean()
    ## If any totalequity is negtive, we return empty and say that this stock is garbage and will probably go bankrupt
    if((raw_dfs["BALANCE_SHEET"]["commonStockTotalEquity"] < 0).sum() > 0):
        print("This stock might go bankrupt due toa  negative totalEquity")
        return empty

    #longTermDebt
    if (raw_dfs["BALANCE_SHEET"]["longTermDebt"].isna().sum() > 2):
        return empty
    else:
        raw_dfs["BALANCE_SHEET"]["longTermDebt"] = raw_dfs["BALANCE_SHEET"]["longTermDebt"].fillna(0)
        
    # totalShareholderEquity:
    if (raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"] = pd.concat([raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"].ffill(), raw_dfs["BALANCE_SHEET"]["totalShareholderEquity"].bfill()]).groupby(level=0).mean()

    # totalAssets:
    if (raw_dfs["BALANCE_SHEET"]["totalAssets"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["totalAssets"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["totalAssets"] = pd.concat([raw_dfs["BALANCE_SHEET"]["totalAssets"].ffill(), raw_dfs["BALANCE_SHEET"]["totalAssets"].bfill()]).groupby(level=0).mean()

    # operatingIncome:
    if (raw_dfs["INCOME_STATEMENT"]["operatingIncome"].isna().sum() > 1):
        return empty
    elif(raw_dfs["INCOME_STATEMENT"]["operatingIncome"].isna().sum() == 1):
        raw_dfs["INCOME_STATEMENT"]["operatingIncome"] = pd.concat([raw_dfs["INCOME_STATEMENT"]["operatingIncome"].ffill(), raw_dfs["INCOME_STATEMENT"]["operatingIncome"].bfill()]).groupby(level=0).mean()
    
    # totalRevenue
    if (raw_dfs["INCOME_STATEMENT"]["totalRevenue"].isna().sum() > 2):
        return empty
    else:
        raw_dfs["INCOME_STATEMENT"]["totalRevenue"] = raw_dfs["INCOME_STATEMENT"]["totalRevenue"].fillna(0)

    # netIncomeApplicableToCommonShares:
    if (raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"].isna().sum() > 1):
        return empty
    elif(raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"].isna().sum() == 1):
        raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"] = pd.concat([raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"].ffill(), raw_dfs["INCOME_STATEMENT"]["netIncomeApplicableToCommonShares"].bfill()]).groupby(level=0).mean()
    
    # dividendPayout
    raw_dfs["CASH_FLOW"]["dividendPayout"] = raw_dfs["CASH_FLOW"]["dividendPayout"].fillna(0).abs()

    # ebit:
    if (raw_dfs["INCOME_STATEMENT"]["ebit"].isna().sum() > 1):
        return empty
    elif(raw_dfs["INCOME_STATEMENT"]["ebit"].isna().sum() == 1):
        raw_dfs["INCOME_STATEMENT"]["ebit"] = pd.concat([raw_dfs["INCOME_STATEMENT"]["ebit"].ffill(), raw_dfs["INCOME_STATEMENT"]["ebit"].bfill()]).groupby(level=0).mean()

    # cashAndShortTermInvestments:
    if (raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"] = pd.concat([raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"].ffill(), raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"].bfill()]).groupby(level=0).mean()
    
    raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"] = raw_dfs["BALANCE_SHEET"]["cashAndShortTermInvestments"].abs()

    # shortTermDebt
    raw_dfs["BALANCE_SHEET"]["shortTermDebt"] = raw_dfs["BALANCE_SHEET"]["shortTermDebt"].fillna(0).abs()

    # totalCurrentLiabilities
    if (raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"] = pd.concat([raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"].ffill(), raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"].bfill()]).groupby(level=0).mean()

        raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"] = raw_dfs["BALANCE_SHEET"]["totalCurrentLiabilities"].abs()
    
    # totalCurrentAssets
    if (raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"] = pd.concat([raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"].ffill(), raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"].bfill()]).groupby(level=0).mean()

        raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"] = raw_dfs["BALANCE_SHEET"]["totalCurrentAssets"].abs()

    # inventory
    if (raw_dfs["BALANCE_SHEET"]["inventory"].isna().sum() > 1):
        return empty
    elif(raw_dfs["BALANCE_SHEET"]["inventory"].isna().sum() == 1):
        raw_dfs["BALANCE_SHEET"]["inventory"] = pd.concat([raw_dfs["BALANCE_SHEET"]["inventory"].ffill(), raw_dfs["BALANCE_SHEET"]["inventory"].bfill()]).groupby(level=0).mean()

        raw_dfs["BALANCE_SHEET"]["inventory"] = raw_dfs["BALANCE_SHEET"]["inventory"].abs()

    return raw_dfs

# Test the clean df
def test_clean_df(clean_df):
    return clean_df

# REFACTORS ANNUAL DATA FOR A GIVEN COMPANY
def refactor_anual_company_data(symbol, verbose=True):
    # this loop extracts, cleans and arranges all raw dataframes in a dictionary
    raw_dfs = {} 
    dataset_names = API_FUNCTIONS
    dataset_names[0] = 'PRICES'
    for f, i in zip(dataset_names, range(len(dataset_names))):
        file_path = "./companies/" + symbol + "/" + 'Y_' + symbol + '_' + f + ".csv"
        if not(os.path.exists(file_path)):
            if(verbose):
                print(file_path + " does not exists. Download needed?")

            return False
    
        df = pd.read_csv(file_path)

        if(df.empty):
            if(verbose):
                print(f + " is empty")
            
            return False

        if not(f == "PRICES"):    # drop useless column
            df = df.drop(df.columns[0], axis=1)

        df = df.replace(["None", ""], np.nan)

        # Finally convert the rest of the columns to numeric
        selection = [x for x in df.columns if x not in ["Date" ,"fiscalDateEnding", "reportedDate", "reportedCurrency", "Unnamed: 0"]]
        df[selection] = df[selection].apply(pd.to_numeric)

        raw_dfs[f] = df
    

    # The BALANCE_SHEET will be our reference, so if its not exactly of shape             DATA_TIME_STEP_COUNT, or all the results are in dollars we will return False
    if(raw_dfs["BALANCE_SHEET"].shape[0] < MINIMUN_DATA_YEARS):
        if (verbose):
            print("BALANCE_SHEET does not have " + str(MINIMUN_DATA_YEARS) + " rows")
        return False
        # Check all dataframes are of same length

    # Adjust Prices to use same rows as BALANCE_SHEET
    dates = raw_dfs["BALANCE_SHEET"]["year"].to_list()
    raw_dfs["PRICES"] = raw_dfs["PRICES"][raw_dfs["PRICES"]["year"].isin(dates)].reset_index().drop(columns=['index'])

    # Now we do inverse operation: let the dfs match the PRICES
    dates = raw_dfs["PRICES"]["year"].to_list()
    for name, df in raw_dfs.items():
        if (name != "PRICES"):
            raw_dfs[name] = raw_dfs[name][raw_dfs[name]["year"].isin(dates)].reset_index().drop(columns=['index'])

    # Transform some nan values 
    raw_dfs = fill_raw_data(raw_dfs)   
    if(not raw_dfs):
        if(verbose):
            print("The company has alarming results or the data is incorrect. Aborting...")
        
        return False

    # new clean_df where we will insert the desired data
    clean_df = pd.DataFrame()

    # add the date column to new df
    clean_df["year"] = raw_dfs["BALANCE_SHEET"]["year"]

    # add the nextYearGrowth column(in %)
    clean_df["nextYearGrowth"] = np.nan
    for i in range(clean_df.shape[0] - 1, 0, -1):
        clean_df["nextYearGrowth"][i] = ((raw_dfs["PRICES"]["averagePrice"][i - 1] - raw_dfs["PRICES"]["averagePrice"][i])/(raw_dfs["PRICES"]["averagePrice"][i]))

    # now check all the df are of the same length
    for k, v in raw_dfs.items():
        if(v.shape[0] != raw_dfs["BALANCE_SHEET"].shape[0]):
            if (verbose):
                print("Dataframe sizes do not matach")

            return False

    # Check that all the results were given in USD
    currency_values = raw_dfs["BALANCE_SHEET"]["reportedCurrency"].unique()
    if(len(currency_values) != 1 or currency_values[0] != "USD"):
        if (verbose):
            print("The results report are not exclusively in USD")

        return False

    # Refactor the cashFlow dividentPayout:
    raw_dfs["CASH_FLOW"]["dividendPayout"] = raw_dfs["CASH_FLOW"]["dividendPayout"].fillna(0)

    # add the averagePrice column to the clean DF:
    clean_df['averagePrice'] = raw_dfs["PRICES"]["averagePrice"]

    # fill the new df with empty columns:
    for ratio_name in RATIOS_FUNCTIONS.keys():
        clean_df[ratio_name] = np.nan

    # finally calculate all the ratios and fill the clean_df
    for i in range(raw_dfs["BALANCE_SHEET"].shape[0]):
        for ratio_name, ratio_function in RATIOS_FUNCTIONS.items():
            clean_df[ratio_name][i] = ratio_function(raw_dfs, i)

    # Add the EV to the clean frame
    # clean_df["enterpriseValue"] = np.nan
    clean_df.insert(loc=1, column='enterpriseValue', value=np.nan)
    for i in range(clean_df.shape[0]):
        clean_df["enterpriseValue"][i] = get_Enterprise_Value(raw_dfs, i)   

    # save it to a csv file
    clean_df.to_csv('./companies/' + symbol + '/Y_' + symbol + '_CLEAN.csv')
    if(verbose):
        display(clean_df)

    return True


In [9]:
refactor_anual_company_data('ABMD')

,year,enterpriseValue,nextYearGrowth,averagePrice,Contributions_To_Debt,ROE,ROA,Sales_Margin,ROIC,ROCE,EV/EBIT,PBV,PER,Dividend_Yield,Treasury_Ratio,Acid_Test,RSCD
0,2020,2.386905e+11,NaN,404.569400,1.200614,0.297194,0.161478,0.260670,1.599398,1.517068,78.984273,247596.472800,88.553817,0.001575,6.108184,7.125000,58.115385
1,2019,1.013647e+11,1.295618,176.235542,1.667844,0.796193,0.311541,0.324684,2.268477,2.119658,24.042870,106798.738250,25.790567,0.003474,5.584650,6.758465,72.689655
2,2018,1.326559e+11,-0.224854,227.357992,1.157111,0.688743,0.271061,0.330451,1.705942,1.740181,38.384243,137778.942950,45.217901,0.002475,6.164788,7.336513,45.473684
3,2017,8.327782e+10,0.509821,150.586017,0.579816,0.406738,0.169292,0.280318,0.971270,1.083669,38.733870,88092.819750,52.876843,0.002963,3.737218,4.256185,2.517564
4,2016,2.526353e+10,1.686408,56.054792,7.373973,0.159274,0.083311,0.149102,9.397727,8.977273,31.979155,30213.532708,49.207708,0.007050,2.142493,2.396852,0.541096


True

In [10]:
df = pd.read_csv('./companies/KO/Y_KO_CLEAN.csv')
cols = df.columns
cols = cols.insert(0, "symbol")
dataset = pd.DataFrame(columns=cols)

for company in os.listdir('./companies'):
    refactor_anual_company_data(company)
    time.sleep(0.01)
    if(os.path.exists('./companies/' + company + '/Y_' + company + '_CLEAN.csv')):
        temp_df = pd.read_csv('./companies/' + company + '/Y_' + company + '_CLEAN.csv')
        temp_df['symbol'] = company
        dataset = dataset.append(temp_df, ignore_index = True)


dataset = dataset.drop(columns=['Unnamed: 0']).dropna().reset_index().drop(columns=['index'])

dataset.to_csv('./pene.csv')
display(dataset)

,symbol,year,enterpriseValue,nextYearGrowth,averagePrice,Contributions_To_Debt,ROE,ROA,Sales_Margin,ROIC,ROCE,EV/EBIT,PBV,PER,Dividend_Yield,Treasury_Ratio,Acid_Test,RSCD
0,NVDA,2019,1.013647e+11,1.295618,176.235542,1.667844,0.796193,0.311541,0.324684,2.268477,2.119658,24.042870,106798.738250,25.790567,0.003474,5.584650,6.758465,72.689655
1,NVDA,2018,1.326559e+11,-0.224854,227.357992,1.157111,0.688743,0.271061,0.330451,1.705942,1.740181,38.384243,137778.942950,45.217901,0.002475,6.164788,7.336513,45.473684
2,NVDA,2017,8.327782e+10,0.509821,150.586017,0.579816,0.406738,0.169292,0.280318,0.971270,1.083669,38.733870,88092.819750,52.876843,0.002963,3.737218,4.256185,2.517564
3,NVDA,2016,2.526353e+10,1.686408,56.054792,7.373973,0.159274,0.083311,0.149102,9.397727,8.977273,31.979155,30213.532708,49.207708,0.007050,2.142493,2.396852,0.541096
4,KO,2018,1.951167e+11,0.160992,42.033892,-0.107806,0.610031,0.077317,0.273104,0.482156,0.341727,21.050459,102.672557,28.085748,0.036767,0.374739,0.953632,0.485034
5,KO,2017,1.871055e+11,0.061145,39.611808,-0.009688,0.078868,0.014199,0.211833,0.229737,0.230192,24.674332,97.319011,137.244759,0.036898,0.564757,1.246231,1.832085
6,KO,2016,1.747983e+11,0.059538,37.385917,-0.052133,0.394738,0.074791,0.206053,0.399758,0.282057,19.708907,92.763806,25.013681,0.037014,0.684080,1.181027,2.081925
7,KO,2015,1.642014e+11,0.087565,34.375792,-0.015613,0.403835,0.081593,0.197047,0.433984,0.346770,15.696526,86.037138,20.599287,0.037913,0.580431,1.132306,2.960940
8,AA,2018,8.717665e+09,-0.447334,42.485000,0.161823,0.046572,0.015497,0.135119,0.597338,1.359956,3.555328,4014.832500,32.118660,0.102993,0.381295,0.859541,19.934959
9,AA,2017,7.429278e+09,0.073712,39.568333,0.213855,0.065740,0.015991,0.114229,0.446763,1.449640,3.686987,3699.639167,26.520711,0.046221,0.417589,0.856396,16.791667


We Finally start with the cool stuff: lets start by getting our dataset:

In [19]:
df = pd.read_csv('./companies/KO/Y_KO_CLEAN.csv')
cols = df.columns
cols = cols.insert(0, "symbol")
dataset = pd.DataFrame(columns=cols)

for company in os.listdir('./companies'):
    refactor_anual_company_data(company, verbose=False)

    if(os.path.exists('./companies/' + company + '/Y_' + company + '_CLEAN.csv')):
        temp_df = pd.read_csv('./companies/' + company + '/Y_' + company + '_CLEAN.csv')
        temp_df['symbol'] = company
        dataset = dataset.append(temp_df, ignore_index = True)


dataset = dataset.drop(columns=['Unnamed: 0']).dropna().reset_index().drop(columns=['index'])

# For the moment we are not going to work with the PBV since this ratio is more relevant to compare only between stock of same industry
dataset = dataset.drop(columns=['PBV', 'RSD'])
dataset

,symbol,year,enterpriseValue,nextYearGrowth,averagePrice,Contributions_To_Debt,ROE,ROA,Sales_Margin,ROIC,ROCE,EV/EBIT,PER,Dividend_Yield,Treasury_Ratio,Acid_Test,RSCD
0,NVDA,2019,1.013647e+11,1.295618,176.235542,1.667844,0.796193,0.311541,0.324684,2.268477,2.119658,24.042870,25.790567,0.003474,5.584650,6.758465,72.689655
1,NVDA,2018,1.326559e+11,-0.224854,227.357992,1.157111,0.688743,0.271061,0.330451,1.705942,1.740181,38.384243,45.217901,0.002475,6.164788,7.336513,45.473684
2,NVDA,2017,8.327782e+10,0.509821,150.586017,0.579816,0.406738,0.169292,0.280318,0.971270,1.083669,38.733870,52.876843,0.002963,3.737218,4.256185,2.517564
3,NVDA,2016,2.526353e+10,1.686408,56.054792,7.373973,0.159274,0.083311,0.149102,9.397727,8.977273,31.979155,49.207708,0.007050,2.142493,2.396852,0.541096
4,KO,2018,1.951167e+11,0.160992,42.033892,-0.107806,0.610031,0.077317,0.273104,0.482156,0.341727,21.050459,28.085748,0.036767,0.374739,0.953632,0.485034
5,KO,2017,1.871055e+11,0.061145,39.611808,-0.009688,0.078868,0.014199,0.211833,0.229737,0.230192,24.674332,137.244759,0.036898,0.564757,1.246231,1.832085
6,KO,2016,1.747983e+11,0.059538,37.385917,-0.052133,0.394738,0.074791,0.206053,0.399758,0.282057,19.708907,25.013681,0.037014,0.684080,1.181027,2.081925
7,KO,2015,1.642014e+11,0.087565,34.375792,-0.015613,0.403835,0.081593,0.197047,0.433984,0.346770,15.696526,20.599287,0.037913,0.580431,1.132306,2.960940
8,AA,2018,8.717665e+09,-0.447334,42.485000,0.161823,0.046572,0.015497,0.135119,0.597338,1.359956,3.555328,32.118660,0.102993,0.381295,0.859541,19.934959
9,AA,2017,7.429278e+09,0.073712,39.568333,0.213855,0.065740,0.015991,0.114229,0.446763,1.449640,3.686987,26.520711,0.046221,0.417589,0.856396,16.791667


Scale the data:

In [20]:
dataset["EV/EBIT"] = dataset["EV/EBIT"].pow(-1)
dataset["PER"] = dataset["PER"].pow(-1)

Define X and y, Split the dataset:

In [21]:
train_dataset = dataset.iloc[:,3:].sample(frac=0.7, random_state=0)
test_dataset = dataset.iloc[:,3:].drop(train_dataset.index)

X_train = train_dataset
X_test = test_dataset

y_train = train_dataset.pop('nextYearGrowth')
y_test = test_dataset.pop('nextYearGrowth')


In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam( )

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


In [ ]:
EPOCHS = 1000

model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x_train, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [14]:
#TODO: deprecated
# def refactor_quarter_company_data(symbol, verbose=True):
#     # this loop extracts, cleans and arranges all raw dataframes in a dictionary
#     raw_dfs = {} 
#     for f, i in zip(API_FUNCTIONS, range(len(API_FUNCTIONS))):
#         file_path = "./companies/" + symbol + "/" + symbol + '_' + f + ".csv"
#         if not(os.path.exists(file_path)):
#             if(verbose):
#                 print(file_path + " does not exists. Download needed?")

#             return False
    
#         df = pd.read_csv(file_path)

#         if(df.empty):
#             if(verbose):
#                 print(f + " is empty")
            
#             return False

#         if not(f == "TIME_SERIES_MONTHLY"):    # drop useless column
#             df = df.drop(df.columns[0], axis=1)

#         # Rename the date labels:
#         if(f == "TIME_SERIES_MONTHLY"):
#             df = df.rename(columns={'Unnamed: 0': 'Date'})
#         else:
#             df = df.rename(columns={'fiscalDateEnding': 'Date'})

#         #Convert all None and NaN items and replace them by Forward Fill method
#         df = df.replace(["None", ""], np.nan).fillna(method='ffill')

#         #convert to datetime
#         df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], yearfirst=True)

#         # Set all dates to the first day of the month
#         df["Date"] = [x - pd.Timedelta(x.day, unit='D') + pd.Timedelta(1, unit='D') for x in df["Date"]] 

#         # Finally convert the rest of the columns to numeric
#         selection = [x for x in df.columns if x not in ["Date" ,"fiscalDateEnding", "reportedDate", "reportedCurrency", "Unnamed: 0"]]
#         df[selection] = df[selection].apply(pd.to_numeric)

#         raw_dfs[f] = df
    

#     # The BALANCE_SHEET will be our reference, so if its not exactly of shape             DATA_TIME_STEP_COUNT, or all the results are in dollars we will return False
#     if(raw_dfs["BALANCE_SHEET"].shape[0] != DATA_TIME_STEP_COUNT):
#         if (verbose):
#             print("BALANCE_SHEET does not have " + str(DATA_TIME_STEP_COUNT) + " rows")
#         return False

#     currency_values = raw_dfs["BALANCE_SHEET"]["reportedCurrency"].unique()
#     if(len(currency_values) != 1 or currency_values[0] != "USD"):
#         if (verbose):
#             print("The results report are not exclusively in USD")

#         return False

#     # Refactor the cashFlow dividentPayout:
#     raw_dfs["CASH_FLOW"]["dividendPayout"] = raw_dfs["CASH_FLOW"]["dividendPayout"].fillna(0)

#     # Change the TIME_SERIES_MONTHLY to match other df dates:
#     selection = raw_dfs['BALANCE_SHEET']['Date'].to_list()
#     raw_dfs["TIME_SERIES_MONTHLY"] = raw_dfs["TIME_SERIES_MONTHLY"].loc[raw_dfs["TIME_SERIES_MONTHLY"]['Date'].isin(selection)]
#     # Reshape the index to make sence
#     raw_dfs["TIME_SERIES_MONTHLY"].index = range(raw_dfs["TIME_SERIES_MONTHLY"].shape[0])

#     # Check if all the data we have accomplishes the MIN_QUARTER_RESULTS_COUT
#     # If it accomplishes the min, but is not exactly 21, we will the remain
#     for name, df in raw_dfs.items():
#         if(df.shape[0] < MIN_QUARTER_RESULTS_COUT):
#             if(verbose):
#                 print("Not enought data for this stock in " + str(name))
        
#             return False

#         # the row count is: MIN_QUARTER_RESULTS_COUT < row_count <                             DATA_TIME_STEP_COUNT, so we need to fill with rows to match exactly                  DATA_TIME_STEP_COUNT
#         elif(df.shape[0] < DATA_TIME_STEP_COUNT):
#             rows_left = DATA_TIME_STEP_COUNT - df.shape[0]
#             for i in range(rows_left):
#                 raw_dfs[name] = raw_dfs[name].append(pd.Series(np.zeros), ignore_index=True)
#                 ##TODO: Fill the date with -3 months from previous row Date

#             # After filling with nan, procceed with 
#             raw_dfs[name] = raw_dfs[name].fillna(method='ffill')


#     # Add an averagePrice column to TIME_SERIES_MONTHLY
#     raw_dfs["TIME_SERIES_MONTHLY"]["averagePrice"] = np.nan
#     raw_dfs["TIME_SERIES_MONTHLY"]["averagePrice"] = (raw_dfs["TIME_SERIES_MONTHLY"]["2. high"] + raw_dfs["TIME_SERIES_MONTHLY"]["3. low"]) / 2

#     # new clean_df where we will insert the desired data
#     clean_df = pd.DataFrame()

#     # add the date column to new df
#     clean_df["Date"] = raw_dfs["BALANCE_SHEET"]["Date"]

#     # add the averagePrice column:
#     clean_df['averagePrice'] = raw_dfs["TIME_SERIES_MONTHLY"]["averagePrice"]

#     # fill the new df with empty columns:
#     for ratio_name in RATIOS_FUNCTIONS.keys():
#         clean_df[ratio_name] = np.nan


#     # finally calculate all the ratios and fill the clean_df
#     for i in range(DATA_TIME_STEP_COUNT):
#         for ratio_name, ratio_function in RATIOS_FUNCTIONS.items():
#             clean_df[ratio_name][i] = ratio_function(raw_dfs, i)

#     # save it to a csv file
    
#     clean_df.to_csv('./companies/' + symbol+ '/' + symbol + '_' + 'CLEAN' + '.csv')
#     display(clean_df)
#     return True
